In [ ]:
from pandas import *

xls = ExcelFile('taiwan_credit_dataset.xls')
df = xls.parse(xls.sheet_names[0])


In [ ]:
import numpy as np
from numpy.linalg import inv

In [ ]:
# i think this is no longer needed
np.random.seed()
training_indices = np.random.choice([i for i in range(1, 30001)], 15000, replace=0)

training_indices.tolist().sort()

# construction of the training and test datasets as dictionaries
# keys are identical to the original file

training_set = {}
test_set     = {}

df.keys()

for key in df.keys():
    training_set[key] = {}
    test_set[key]     = {}
    for index in range(1, 30001):
        if index in training_indices:
            training_set[key][index] = df[key][index]
        else:
            test_set[key][index] = df[key][index]

In [ ]:
wstar_coeff = [1]*(22)
wstar_coeff.append(0)
wstar       = np.array(wstar_coeff)
#wstar_coeff = [np.random.uniform(0, 1) for _ in range(22)]
#wstar_coeff.append(0)
#wstar        = np.array(wstar_coeff)

#wstar = np.array([0.50100192, 0.07380532, 0.60986838, 0.62581584, 0.79904207, 0.59658988, 0.16605999, 0.55851739, 0.43282946, 0.62053332, 0.10610947, 0.76089288, 0.52961132,  0.61544528, 0.42376386, 0.44707219, 0.91981011, 0.24696523, 0.89761642, 0.32054103, 0.14994946, 0.06403403, 0.00])

#print (wstar)

dim = 23

temp1 = np.random.uniform(-1,1,size=(dim,dim))
A1 = (temp1 + temp1.T)/2

temp2 = np.random.uniform(-1,1,size=(dim,dim))
A2 = (temp2 + temp2.T)/2

A1_inv = inv(A1)
A2_inv = inv(A2)

In [ ]:
grA_tot = {} # dictionary with data from group A -- should the SVD be computed from the total population or the training set?
grB_tot = {}

for key in df.keys():
    grA_tot[key] = {}
    grB_tot[key] = {}

    for j in range(1,30001): # this is building the groups for the SVD
        # categorizing: married or not
        #if df['X4'][j] == 1:
        #    grA_tot[key][j] = df[key][j]
        #elif df['X4'][j] == 2:
        #    grB_tot[key][j] = df[key][j]

        # categorizing: <=25 or older
        if df['X5'][j] <= 25:
            grA_tot[key][j] = df[key][j]
        elif df['X5'][j] > 25:
            grB_tot[key][j] = df[key][j]

        # categorizing: grad school+uni vs high school
        #if df['X3'][j] < 2:
        #    grA_tot[key][j] = df[key][j]
        #elif df['X3'][j] >= 3:
        #    grB_tot[key][j] = df[key][j]

        # categorizing: gender
        #if df['X2'][j] == 1:
        #    grA_tot[key][j] = df[key][j]
        #elif df['X2'][j] == 2:
        #    grB_tot[key][j] = df[key][j]

        # features list for grA

dim = 23
lenA = len(grA_tot['Y'])
grA_tot_feat = [[_ for _ in range(0,dim)] for _ in range(0, lenA)]

key_index = 0
for key in grA_tot.keys():
    if key != 'Unnamed: 0' and key != 'Y' and key != 'X4':
        person_index = 0
        for val in iter(grA_tot['Y']):
            grA_tot_feat[person_index][key_index] = grA_tot[key][val]
            grA_tot_feat[person_index][22]        = 1 # in order to account for the intercept of wstar
            person_index += 1

        key_index += 1
print (grA_tot_feat)

# features list for grB

dim = 23
lenB = len(grB_tot['Y'])
grB_tot_feat = [[_ for _ in range(0,dim)] for _ in range(0, lenB)]

key_index = 0
for key in grB_tot.keys():
    if key != 'Unnamed: 0' and key != 'Y' and key != 'X4':
        person_index = 0
        for val in iter(grB_tot['Y']):
            grB_tot_feat[person_index][key_index] = grB_tot[key][val]
            grB_tot_feat[person_index][22]        = 1 # in order to account for the intercept of wstar
            person_index += 1

        key_index += 1



[[20000, 2, 2, 24, 2, 2, -1, -1, -2, -2, 3913, 3102, 689, 0, 0, 0, 0, 689, 0, 0, 0, 0, 1], [100000, 2, 2, 23, 0, -1, -1, 0, 0, -1, 11876, 380, 601, 221, -159, 567, 380, 601, 0, 581, 1687, 1542, 1], [50000, 2, 3, 23, 1, 2, 0, 0, 0, 0, 50614, 29173, 28116, 28771, 29531, 30211, 0, 1500, 1100, 1200, 1300, 1100, 1], [20000, 1, 1, 24, 0, 0, 2, 2, 2, 2, 15376, 18010, 17428, 18338, 17905, 19104, 3200, 0, 1500, 0, 1650, 0, 1], [90000, 1, 1, 23, 0, 0, 0, -1, 0, 0, 4744, 7070, 0, 5398, 6360, 8292, 5757, 0, 5398, 1200, 2045, 2000, 1], [50000, 1, 3, 23, 0, 0, 0, 0, 0, 0, 47620, 41810, 36023, 28967, 29829, 30046, 1973, 1426, 1001, 1432, 1062, 997, 1], [60000, 2, 2, 22, 0, 0, 0, 0, 0, -1, 15054, 9806, 11068, 6026, -28335, 18660, 1500, 1518, 2043, 0, 47671, 617, 1], [50000, 1, 1, 25, 1, -1, -1, -2, -2, -2, 0, 780, 0, 0, 0, 0, 780, 0, 0, 0, 0, 0, 1], [70000, 2, 1, 25, 0, 0, 0, 0, 0, 0, 67521, 66999, 63949, 63699, 64718, 65970, 3000, 4500, 4042, 2500, 2800, 2500, 1], [10000, 1, 2, 22, 0, 0, 0, 0, 0, 0, 

In [ ]:
grA_tot_feat = np.array(grA_tot_feat)
grB_tot_feat = np.array(grB_tot_feat)

#print (grA_tot_feat)

# SVD to help finding the projection matrices
uA, sA, vA = np.linalg.svd(grA_tot_feat, full_matrices=True)
uB, sB, vB = np.linalg.svd(grB_tot_feat, full_matrices=True)

In [ ]:
# simplifying the numbers that we carry around

#print ("Projection")
print (np.linalg.norm(vA))
print ()
print (np.linalg.norm(vB))
print ()

Pi_1 = np.matmul(vA[:, :15], np.transpose(vA[:,:15]))
Pi_2 = np.matmul(vB[:, :15], np.transpose(vB[:,:15]))

print (np.linalg.norm(Pi_1))
print ()
print (np.linalg.norm(Pi_2))
print ()

for i in range(dim):
  for j in range(dim):
    if np.abs(Pi_1[i][j]) < 0.001:
      Pi_1[i][j] = 0
    if np.abs(Pi_2[i][j]) < 0.001:
      Pi_2[i][j] = 0
    #if Pi_1[i][j] < 0:
    #  print ("1: (%d, %d)"%(i,j))
    #if Pi_2[i][j] < 0:
    #  print ("2: (%d, %d)"%(i,j))

#minPi1 = np.min(Pi_1)
#minPi2 = np.min(Pi_2)

#for i in range(dim):
#  for j in range(dim):
#    Pi_1[i][j] += np.abs(minPi1)
#    Pi_2[i][j] += np.abs(minPi2)

#Pi_1n = Pi_1/np.linalg.norm(Pi_1)
#Pi_2n = Pi_2/np.linalg.norm(Pi_2)

#Pi_1 = Pi_1n
#Pi_2 = Pi_2n

#print (np.linalg.norm(Pi_1))
#print ()
#print (np.linalg.norm(Pi_2))
#print ()

4.795831523312721

4.795831523312721

3.872983346207419

3.872983346207418



In [ ]:
# definition of cost matrices
# for now, they are the identity matrix

#diag = [1]*dim
#A1 = np.diag(diag)
#A2 = np.diag(diag)

# inverse matrices

#A1_inv = inv(A1)
#A2_inv = inv(A2)



In [ ]:
# computation of the numerator for w
A1Pi1  = np.matmul(A1_inv, Pi_1)
A2Pi2  = np.matmul(A2_inv, Pi_2)
Anum   = A1Pi1 + A2Pi2
Anum_T = np.transpose(Anum)
num    = np.matmul(Anum_T, wstar)

# computation of the denominator for w
denom  = np.linalg.norm(num)

# final computation of vw
w      = 1.0*num/denom

#print ("Outcome-decision rule deployed by principal: ")
#print (w)


# TOTAL outcome improvement for two groups
# The results from here portray what happens in practice for Thm 4.3 (equal total outcome improvement) and Thm 4.2 (do-no-harm)
Delta_1 = np.matmul(A1Pi1,w)
I1      = np.matmul(np.transpose(Delta_1), wstar)
print ("Total outcome improvement for group 1: %f\n"%I1)

Delta_2 = np.matmul(A2Pi2, w)
I2      = np.matmul(np.transpose(Delta_2), wstar)
print ("Total outcome improvement for group 2: %f\n"%I2)

temp = np.matmul(Pi_1 - Pi_2, wstar)
overlap_proxy = np.sqrt(np.matmul(np.transpose(temp),temp))
print ("overlap proxy: %f \n"%overlap_proxy)

#print ("Norm of w's num: %f\n"%denom)

# per unit improvements
w1   = np.matmul(Pi_1, w)
w1_n = w1/np.linalg.norm(w1)

Delta_1_u = np.matmul(A1Pi1,w1_n)
uI1       = np.matmul(np.transpose(Delta_1_u), wstar)
print ("Per-unit outcome improvement for group 1: %f\n"%uI1)

w2   = np.matmul(Pi_2, w)

#print (w2)

w2_n = w2/np.linalg.norm(w2)

#print (w2_n)

Delta_2_u = np.matmul(A2Pi2,w2_n)

#print (A2Pi2)

uI2       = np.matmul(np.transpose(Delta_2_u), wstar)
print ("Per-unit outcome improvement for group 2: %f\n"%uI2)


Total outcome improvement for group 1: 13.961643

Total outcome improvement for group 2: 2.738583

overlap proxy: 1.786239 

Per-unit outcome improvement for group 1: 14.029391

Per-unit outcome improvement for group 2: 2.805792



In [ ]:
# computation of OPTIMAL TOTAL outcome improvement for the two groups
# group 1
num_w1_opt   = np.matmul(np.transpose(A1Pi1), wstar)
denom_w1_opt = np.linalg.norm(num_w1_opt)
w1_opt       = 1.0*num_w1_opt/denom_w1_opt

Delta_1_w1_opt = np.matmul(A1Pi1, w1_opt)
I1_opt         = np.matmul(np.transpose(Delta_1_w1_opt), wstar)
print ("Optimal total outcome improvement for group 1: %f\n"%I1_opt)

# group 2
num_w2_opt   = np.matmul(np.transpose(A2Pi2), wstar)
denom_w2_opt = np.linalg.norm(num_w2_opt)
w2_opt       = 1.0*num_w2_opt/denom_w2_opt

Delta_2_w2_opt = np.matmul(A2Pi2, w2_opt)
I2_opt         = np.matmul(np.transpose(Delta_2_w2_opt), wstar)
print ("Optimal total outcome improvement for group 2: %f\n"%I2_opt)


Optimal total outcome improvement for group 1: 14.794895

Optimal total outcome improvement for group 2: 5.609035



In [ ]:
# projection computation; TODO: check with the others
# problem here is that the matrix may (and in general will) not be orthonormal. So i'm not sure what to do about this.

reps = 10000
tot  = 0
#Pi_1 = np.array(np.diag(sA))
#Pi_2 = np.array(np.diag(sB))
for r in range(reps):
    randx = [np.random.uniform(-1,1) for _ in range(dim)] # random point to be projected to estimate the orthogonality of the two subspaces
    randx = np.array(randx)
    # compute projections of randx in the subspaces
    # if we agree with this method, then i should run this in multiple samples so as to have conf bounds too
    proj1 = np.matmul(Pi_1, randx)/np.linalg.norm(np.matmul(Pi_1, randx))
    proj2 = np.matmul(Pi_2, randx)/np.linalg.norm(np.matmul(Pi_2, randx))
    tot  += np.dot(proj1, proj2)

alignment = 1.0*tot/reps

print ("Alignment of the two subspaces: %f"%alignment)

Alignment of the two subspaces: 0.810226
